# Preprocessing

This jupyter notebook is to preprocess the data from MIMIC and REFLACX to form a dataframe `reflacx_with_clinical.csv` in this repository for training purpose. In the preprocessing phase, we use the metadata table from *REFLACX* dataset as the main table to *left join* three other tables that contian the image and clinical data we want:

- Metadata table from CXR, this data table contain the information about the CXR images, such as height, width and `subject_id`.
- Patient table from MIMIC-IV - CORE, this table contain the information we want, inlcluding gender and age.
- Triage table from MIMIC-IV ED, this table contain the patients' health condition while in emergency department.

Moreover, this notebook also perform data cleaning and filling missing value.

At the end of the notebook, a `reflacx_with_clinical.csv` will be exported to the root of the working directory, so the dataloader can load this file to get training data.

In [1]:
import pandas as pd
import json
from data.paths import TabularDataPaths

from utils import print as print_f
from utils.preprocessing import get_split_list

In [2]:
## Define your MIMIC path here.
XAMI_MIMIC_PATH = "D:\XAMI-MIMIC"
XAMI_MIMIC_PATH_str = "{XAMI_MIMIC_PATH}"


In [3]:
reflacx_meta_df = pd.read_csv(
    TabularDataPaths.SpreadSheet.get_sreadsheet(
        XAMI_MIMIC_PATH, TabularDataPaths.SpreadSheet.REFLACX.metadata,
    )
)
reflacx_meta_df.head(5)


,Unnamed: 0,id,split,eye_tracking_data_discarded,image,dicom_id,subject_id,image_size_x,image_size_y,Airway wall thickening,...,Support devices,Wide mediastinum,Abnormal mediastinal contour,Acute fracture,Enlarged hilum,Hiatal hernia,High lung volume / emphysema,Interstitial lung disease,Lung nodule or mass,Pleural abnormality
0,0,P102R108387,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,P102R379837,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,P102R558314,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,4.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,P102R765317,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,...,True,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,P102R915878,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Define columns in their own categories

In [4]:
labels_cols = [
    "Airway wall thickening",
    "Atelectasis",
    "Consolidation",
    "Enlarged cardiac silhouette",
    "Fibrosis",
    "Fracture",
    "Groundglass opacity",
    "Pneumothorax",
    "Pulmonary edema",
    "Wide mediastinum",
    "Abnormal mediastinal contour",
    "Acute fracture",
    "Enlarged hilum",
    "Quality issue",
    "Support devices",
    "Hiatal hernia",
    "High lung volume / emphysema",
    "Interstitial lung disease",
    "Lung nodule or mass",
    "Pleural abnormality",
]


## inlcude the repetitive columns.
all_disease_cols = [
    "Airway wall thickening",
    "Atelectasis",
    "Consolidation",
    "Emphysema",
    "Enlarged cardiac silhouette",
    "Fibrosis",
    "Fracture",
    "Groundglass opacity",
    "Mass",
    "Nodule",
    "Other",
    "Pleural effusion",
    "Pleural thickening",
    "Pneumothorax",
    "Pulmonary edema",
    "Wide mediastinum",
    "Abnormal mediastinal contour",
    "Acute fracture",
    "Enlarged hilum",
    "Hiatal hernia",
    "High lung volume / emphysema",
    "Interstitial lung disease",
    "Lung nodule or mass",
    "Pleural abnormality",
]


## Filling NAN value.

In [5]:
def checkMissingValue(df):
    print(
        [
            (col, sum(df[col].isna()))
            for col in df.columns
            if df[col].isna().any()
        ]
    )

In [6]:
len(reflacx_meta_df)

3052

In [7]:
2757 + 295

3052

In [8]:
# Checking the columns that contian nan values.
print_f.print_title("Columns have NaN value in REFLACX metadata")
checkMissingValue(reflacx_meta_df)

====================Columns have NaN value in REFLACX metadata====================
[('Airway wall thickening', 2757), ('Emphysema', 2757), ('Fibrosis', 2757), ('Fracture', 2757), ('Mass', 2757), ('Nodule', 2757), ('Other', 2839), ('Pleural effusion', 2757), ('Pleural thickening', 2757), ('Quality issue', 2757), ('Wide mediastinum', 2757), ('Abnormal mediastinal contour', 295), ('Acute fracture', 295), ('Enlarged hilum', 295), ('Hiatal hernia', 295), ('High lung volume / emphysema', 295), ('Interstitial lung disease', 295), ('Lung nodule or mass', 295), ('Pleural abnormality', 295)]


In [9]:
# Filling nan for numerical cols.
reflacx_meta_df[all_disease_cols] = reflacx_meta_df[all_disease_cols].fillna(0)

# Folling nan for boolean cols.
reflacx_meta_df[["Quality issue", "Support devices"]] = reflacx_meta_df[
    ["Quality issue", "Support devices"]
].fillna(False)



## Checking again that all columns don't contain nan value.

In [10]:
print_f.print_title("Columns have missing value in REFLACX metadata.")

checkMissingValue(reflacx_meta_df)


====================Columns have missing value in REFLACX metadata.====================
[]


## Relabel repetitive columns

In [11]:
reflacx_meta_df["Lung nodule or mass"] = reflacx_meta_df[
    ["Lung nodule or mass", "Mass", "Nodule"]
].max(axis=1)
del reflacx_meta_df["Mass"]
del reflacx_meta_df["Nodule"]

reflacx_meta_df["High lung volume / emphysema"] = reflacx_meta_df[
    ["High lung volume / emphysema", "Emphysema"]
].max(axis=1)
del reflacx_meta_df["Emphysema"]


reflacx_meta_df["Pleural abnormality"] = reflacx_meta_df[
    ["Pleural abnormality", "Pleural thickening", "Pleural effusion"]
].max(axis=1)
del reflacx_meta_df["Pleural thickening"]
del reflacx_meta_df["Pleural effusion"]



## Load the dataframe that we wanna join with relfacx metaadta

In [12]:
cxr_meta_df = pd.read_csv(
    TabularDataPaths.SpreadSheet.get_sreadsheet(
        XAMI_MIMIC_PATH, TabularDataPaths.SpreadSheet.cxr_meta
    )
)
core_patients_df = pd.read_csv("D:\mimic-iv-1.0\mimic-iv-1.0\core\patients.csv")
ed_triage_df = pd.read_csv("D:\mimic-iv-ed-1.0\mimic-iv-ed-1.0\ed\\triage.csv")



## Join dataframes

In [13]:
merged_df = (
    reflacx_meta_df.merge(
        cxr_meta_df,
        "left",
        left_on="dicom_id",
        right_on="dicom_id",
        suffixes=("", "cxr"),
    )
    .merge(core_patients_df, "left", on="subject_id", suffixes=("", "_patient"))
    .merge(ed_triage_df, "left", on="stay_id", suffixes=("", "_triage"))
)



## Only keep the cases with identified `stay_id`

In [14]:
# ! If I tracking data is needed, we need to uncomment this one.
merged_df = merged_df[~merged_df["eye_tracking_data_discarded"]]

## only get the one with `stay_id`, so we can link to triage.csv
merged_df = merged_df[~merged_df["stay_id"].isna()]
merged_df["stay_id"] = merged_df["stay_id"].astype(int)



##  Calculate the age according to the time when the CXR taken.

In [15]:
merged_df["age"] = merged_df["anchor_age"] + (
    merged_df["StudyDate"].apply(lambda x: int(str(x)[:4])) - merged_df["anchor_year"]
)

## Generating path for *CXR image* and *bounding ellipses*.
We do this so we can locat them easily during training. When we load the data, we need to replace the `XAMI_MIMIC_PATH` in the path string to get the correct path to desired files.

In [16]:
## Calculate the age according to the time when the CXR taken.
merged_df["image_path"] = (
    f"{XAMI_MIMIC_PATH_str}\patient_"
    + merged_df["subject_id"].astype(str)
    + "\CXR-JPG\s"
    + merged_df["study_id"].astype(str)
    + "\\"
    + merged_df["dicom_id"].astype(str)
    + ".jpg"
)
merged_df["anomaly_location_ellipses_path"] = (
    f"{XAMI_MIMIC_PATH_str}\patient_"
    + merged_df["subject_id"].astype(str)
    + "\REFLACX\\"
    + merged_df["id"].astype(str)
    + "\\anomaly_location_ellipses.csv"
)



# features filtering
By indentified our needed columns, we can filter out the features we don't need.

In [17]:
needed_columns = [
    # ids
    "id",
    "dicom_id",
    "subject_id",
    "stay_id",
    "study_id",
    # image meta
    "image_path",
    "ViewPosition",
    "image_size_x",
    "image_size_y",
    "anomaly_location_ellipses_path",
    # patients
    "age",
    "gender",
    # triage
    "temperature",
    "heartrate",
    "resprate",
    "o2sat",
    "sbp",
    "dbp",
    "pain",
    "acuity",
    # labels
    "Airway wall thickening",
    "Atelectasis",
    "Consolidation",
    "Enlarged cardiac silhouette",
    "Fibrosis",
    "Groundglass opacity",
    "Other",
    "Pneumothorax",
    "Pulmonary edema",
    "Quality issue",
    "Support devices",
    "Wide mediastinum",
    "Abnormal mediastinal contour",
    "Acute fracture",
    "Enlarged hilum",
    "Hiatal hernia",
    "High lung volume / emphysema",
    "Interstitial lung disease",
    "Lung nodule or mass",
    "Pleural abnormality",
    'Fracture' # don't have any positive case
]

merged_df = merged_df[needed_columns]


# Filling NaN value in the merged table.
Previously, we have filled the missing value for `reflacx_meta_df`. However, more features are introduced while we added more columns thorugh left-joining other tables. Therefore, we need to replace the missing value that come from other table.

In [18]:
# Checking the columns that contian nan values (from other joined table).
print_f.print_title("Columns have missing value in merged_df")
checkMissingValue(merged_df)


====================Columns have missing value in merged_df====================
[('temperature', 75), ('heartrate', 61), ('resprate', 69), ('o2sat', 65), ('sbp', 62), ('dbp', 64), ('pain', 83), ('acuity', 24)]


### Filling NaN value by mean

In [19]:
# filling the nan value by mean

for col in merged_df.columns:
    if merged_df[col].isna().any():
        print(f"filling missing value for {col}")
        merged_df[col] = merged_df[col].fillna(round(merged_df[~merged_df[col].isna()][col].mean(), 1))


filling missing value for temperature
filling missing value for heartrate
filling missing value for resprate
filling missing value for o2sat
filling missing value for sbp
filling missing value for dbp
filling missing value for pain
filling missing value for acuity


## Split them into train, val and test set.

In [20]:
merged_df['split'] = get_split_list(len(merged_df))

## The end of the dataframe looks like this: 

In [21]:
len(merged_df)

670

In [22]:
merged_df.head(5)

,id,dicom_id,subject_id,stay_id,study_id,image_path,ViewPosition,image_size_x,image_size_y,anomaly_location_ellipses_path,...,Abnormal mediastinal contour,Acute fracture,Enlarged hilum,Hiatal hernia,High lung volume / emphysema,Interstitial lung disease,Lung nodule or mass,Pleural abnormality,Fracture,split
0,P102R108387,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002,55032240,{XAMI_MIMIC_PATH}\patient_18111516\CXR-JPG\s55...,AP,2544,3056,{XAMI_MIMIC_PATH}\patient_18111516\REFLACX\P10...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
1,P102R379837,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002,55032240,{XAMI_MIMIC_PATH}\patient_18111516\CXR-JPG\s55...,AP,2544,3056,{XAMI_MIMIC_PATH}\patient_18111516\REFLACX\P10...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
2,P102R558314,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002,55032240,{XAMI_MIMIC_PATH}\patient_18111516\CXR-JPG\s55...,AP,2544,3056,{XAMI_MIMIC_PATH}\patient_18111516\REFLACX\P10...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
3,P102R765317,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002,55032240,{XAMI_MIMIC_PATH}\patient_18111516\CXR-JPG\s55...,AP,2544,3056,{XAMI_MIMIC_PATH}\patient_18111516\REFLACX\P10...,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,train
4,P102R915878,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002,55032240,{XAMI_MIMIC_PATH}\patient_18111516\CXR-JPG\s55...,AP,2544,3056,{XAMI_MIMIC_PATH}\patient_18111516\REFLACX\P10...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train


In [36]:
from collections import OrderedDict

count_map = dict((merged_df[labels_cols]>0).sum(axis=0))
OrderedDict({k: v for k, v in sorted(count_map.items(), key=lambda item: item[1])})

OrderedDict([('Fracture', 0),
             ('Fibrosis', 1),
             ('Pneumothorax', 2),
             ('Wide mediastinum', 3),
             ('Quality issue', 3),
             ('Airway wall thickening', 4),
             ('Acute fracture', 4),
             ('Hiatal hernia', 7),
             ('Interstitial lung disease', 11),
             ('Enlarged hilum', 12),
             ('High lung volume / emphysema', 19),
             ('Abnormal mediastinal contour', 20),
             ('Lung nodule or mass', 42),
             ('Groundglass opacity', 61),
             ('Pulmonary edema', 85),
             ('Consolidation', 112),
             ('Pleural abnormality', 138),
             ('Atelectasis', 143),
             ('Enlarged cardiac silhouette', 154),
             ('Support devices', 189)])

## Output the preprocessed data to `reflacx_with_clinical.csv`.
We can export the preprocessed data to the root of local working, so other dataloaders can load them while training. 

In [24]:
merged_df.to_csv("reflacx_with_clinical.csv")

# Create another Dataframe for unique `dicom_id`

In [25]:
u_idxs = [merged_df.dicom_id.eq(u_id).idxmax() for u_id in merged_df.dicom_id.unique()]

In [26]:
u_df = merged_df.loc[u_idxs]
u_df['split'] = get_split_list(len(u_df))

## Remap the bounding bxoes to a single CXR image

In [28]:
all_bbox_paths = []

for idx, row in u_df.iterrows():
    bbox_paths = list(merged_df[merged_df.dicom_id.eq(row['dicom_id'])]['anomaly_location_ellipses_path'])
    all_bbox_paths.append(json.dumps(bbox_paths))

u_df['bbox_paths'] = all_bbox_paths

In [35]:
from collections import OrderedDict

count_map = dict((u_df[labels_cols] >0 ).sum(axis=0))
OrderedDict({k: v for k, v in sorted(count_map.items(), key=lambda item: item[1])})

OrderedDict([('Fibrosis', 0),
             ('Fracture', 0),
             ('Wide mediastinum', 0),
             ('Airway wall thickening', 1),
             ('Quality issue', 1),
             ('Pneumothorax', 2),
             ('Acute fracture', 4),
             ('Hiatal hernia', 6),
             ('Interstitial lung disease', 9),
             ('Enlarged hilum', 12),
             ('High lung volume / emphysema', 15),
             ('Abnormal mediastinal contour', 17),
             ('Lung nodule or mass', 35),
             ('Groundglass opacity', 53),
             ('Pulmonary edema', 71),
             ('Consolidation', 101),
             ('Atelectasis', 117),
             ('Pleural abnormality', 117),
             ('Enlarged cardiac silhouette', 127),
             ('Support devices', 169)])

In [32]:
len(u_df['Enlarged cardiac silhouette'])

590

In [30]:
u_df.to_csv('reflacx_u_df.csv')